<a href="https://colab.research.google.com/github/ChristopherMajor/DS-Unit-2-Applied-Modeling/blob/master/module1-define-ml-problems/Untitled1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
df = pd.read_csv('mushrooms.csv')
df.head()

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,stalk-root,stalk-surface-above-ring,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,e,e,s,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,e,c,s,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,e,c,s,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,e,e,s,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,t,e,s,s,w,w,p,w,o,e,n,a,g


In [0]:
%%capture
import sys

if 'google.colab' in sys.modules:
    # Install packages in Colab
    !pip install category_encoders==2.*

In [0]:
#target will be class
target = 'class'

In [0]:
#this is a classification
#checking disctribution of target
df['class'].value_counts(normalize=True)
#52% edible, 48% poisonous.  not imbalanced. Will be using accuracy evaluation metric

e    0.517971
p    0.482029
Name: class, dtype: float64

In [0]:
#will include all data in random train/val/test split 
#not much cleaning to do, any low variance columns will be sorted out later in 
#random forest model. can't think of any features to engineer. 
#no columns need to be dropped so theres no wrangling to do as of now...

In [0]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=.2, random_state=33)
train.shape, test.shape

((6499, 23), (1625, 23))

In [0]:
train, val = train_test_split(train, test_size=.2, random_state=33)

In [0]:
val.shape, train.shape, test.shape

((1300, 23), (5199, 23), (1625, 23))

In [0]:
X_train=train.drop(columns=target)
X_test =test.drop(columns=target)
y_train = train[target]
y_test = test[target]
X_val = val.drop(columns=target)
y_val= val[target]

In [0]:
y_train


3275    e
0       p
478     e
1963    e
4151    p
       ..
3929    p
4123    p
7948    p
1114    e
249     e
Name: class, Length: 5199, dtype: object

In [0]:
X_train.head()

,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,stalk-root,stalk-surface-above-ring,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
6944,f,y,n,f,s,f,c,n,b,t,?,s,k,w,p,p,w,o,e,w,v,l
7853,k,y,e,f,s,f,c,n,b,t,?,s,s,p,p,p,w,o,e,w,v,l
4750,f,f,y,f,f,f,c,b,g,e,b,k,k,p,b,p,w,o,l,h,v,d
1877,x,f,e,t,n,f,c,b,p,t,b,s,s,w,g,p,w,o,p,k,y,d
5789,b,s,p,t,n,f,c,b,w,e,b,s,s,w,w,p,w,t,p,r,v,m


In [0]:
import category_encoders as ce
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from scipy.stats import randint, uniform
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [0]:
from sklearn.metrics import accuracy_score

pipeline=make_pipeline(
    ce.OrdinalEncoder(),
    RandomForestClassifier(random_state=33)
)

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_val)
print('Validation accuracy:', accuracy_score(y_val, y_pred))

Validation accuracy: 1.0


In [0]:
#trying one hot encoder instead of oridnal.  no improvements
pipeline=make_pipeline(
    ce.OneHotEncoder(),
    RandomForestClassifier(random_state=33)
)

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_val)
print('Validation accuracy:', accuracy_score(y_val, y_pred))

Validation accuracy: 1.0


In [0]:
#okay accuracy of 1. we have some leakage
#time to check feature importances
transformers = make_pipeline(
    ce.OrdinalEncoder()
)

X_train_transformed = transformers.fit_transform(X_train)
X_val_transformed = transformers.transform(X_val)

model = RandomForestClassifier(n_estimators=100, random_state=33, n_jobs=-1, max_depth=2)
model.fit(X_train_transformed, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=2, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=False, random_state=33, verbose=0,
                       warm_start=False)

In [0]:
X_val_transformed.head()

,cap-shape_1,cap-shape_2,cap-shape_3,cap-shape_4,cap-shape_5,cap-shape_6,cap-surface_1,cap-surface_2,cap-surface_3,cap-surface_4,cap-color_1,cap-color_2,cap-color_3,cap-color_4,cap-color_5,cap-color_6,cap-color_7,cap-color_8,cap-color_9,cap-color_10,bruises_1,bruises_2,odor_1,odor_2,odor_3,odor_4,odor_5,odor_6,odor_7,odor_8,odor_9,gill-attachment_1,gill-attachment_2,gill-spacing_1,gill-spacing_2,gill-size_1,gill-size_2,gill-color_1,gill-color_2,gill-color_3,...,stalk-color-below-ring_5,stalk-color-below-ring_6,stalk-color-below-ring_7,stalk-color-below-ring_8,stalk-color-below-ring_9,veil-type_1,veil-color_1,veil-color_2,veil-color_3,veil-color_4,ring-number_1,ring-number_2,ring-number_3,ring-type_1,ring-type_2,ring-type_3,ring-type_4,ring-type_5,spore-print-color_1,spore-print-color_2,spore-print-color_3,spore-print-color_4,spore-print-color_5,spore-print-color_6,spore-print-color_7,spore-print-color_8,spore-print-color_9,population_1,population_2,population_3,population_4,population_5,population_6,habitat_1,habitat_2,habitat_3,habitat_4,habitat_5,habitat_6,habitat_7
4786,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,1,0,1,0,0,0,0,...,0,1,0,0,0,1,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0
4695,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,1,0,1,0,0,0,0,...,0,1,0,0,0,1,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0
383,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,1,1,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
6924,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,1,...,0,0,0,0,0,1,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
2651,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,1,0,1,0,1,0,0,...,0,0,0,0,0,1,1,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0


In [0]:
!pip install eli5

     |████████████████████████████████| 112kB 5.2MB/s 


In [0]:
import eli5
from eli5.sklearn import PermutationImportance

permuter = PermutationImportance(
    model,
    scoring='accuracy',
    n_iter=3,
    random_state=33
)

permuter.fit(X_val_transformed, y_val)

PermutationImportance(cv='prefit',
                      estimator=RandomForestClassifier(bootstrap=True,
                                                       ccp_alpha=0.0,
                                                       class_weight=None,
                                                       criterion='gini',
                                                       max_depth=2,
                                                       max_features='auto',
                                                       max_leaf_nodes=None,
                                                       max_samples=None,
                                                       min_impurity_decrease=0.0,
                                                       min_impurity_split=None,
                                                       min_samples_leaf=1,
                                                       min_samples_split=2,
                                                       min_weight_fract

In [0]:
feature_names = X_val.columns.tolist()
pd.Series(permuter.feature_importances_, feature_names).sort_values()

stalk-color-above-ring     -0.002564
stalk-shape                -0.000769
stalk-surface-above-ring   -0.000513
cap-shape                   0.000000
veil-type                   0.000000
gill-attachment             0.000256
stalk-color-below-ring      0.000256
ring-type                   0.000513
veil-color                  0.000769
stalk-surface-below-ring    0.001026
cap-color                   0.001795
stalk-root                  0.003077
ring-number                 0.003590
spore-print-color           0.004359
bruises                     0.006154
gill-color                  0.006410
cap-surface                 0.006410
habitat                     0.006410
gill-spacing                0.007179
population                  0.008974
gill-size                   0.015128
odor                        0.049744
dtype: float64

In [0]:
eli5.show_weights(
    permuter,
    top=None,
    feature_names=feature_names
)

Weight,Feature
0.0497 ± 0.0064,odor
0.0151 ± 0.0032,gill-size
0.0090 ± 0.0048,population
0.0072 ± 0.0029,gill-spacing
0.0064 ± 0.0048,habitat
0.0064 ± 0.0040,cap-surface
0.0064 ± 0.0026,gill-color
0.0062 ± 0.0022,bruises
0.0044 ± 0.0073,spore-print-color
0.0036 ± 0.0007,ring-number


In [0]:
X_train['cap-shape'].value_counts()

x    2294
f    2021
k     564
b     292
s      25
c       3
Name: cap-shape, dtype: int64